In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last' #all last

import pandas as pd
import numpy as np
import os, sys
sys.path.append('F:\my_documnet_F')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
from importlib import reload
plt=reload(plt)
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import datetime
import seaborn as sns
from scipy import sparse
# import mglearn

import graphviz
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re, logging
from logging.handlers import RotatingFileHandler
import time

import warnings
warnings.filterwarnings("ignore")

#Auto reloads notebook when changes are made
%reload_ext autoreload
%autoreload 2
import matplotlib.ticker as ticker

import imp
import Library.Rawdata_stack as rawstack
imp.reload(rawstack)

<module 'Library.Rawdata_stack' from 'F:\\my_documnet_F\\Library\\Rawdata_stack.py'>

In [65]:
# brew 로 설치된 chromedriver의 path (Mac)
# path = "./chromedriver"

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
path = "F:/my_documnet_F/crawring/chromedriver/chromedriver.exe"

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(path)

# 카카오 지도에 접속합니다
driver.get(source_url)

# 검색창에 검색어를 입력합니다
searchbox = driver.find_element(By.XPATH, "//input[@id='search.keyword.query']")
searchbox.send_keys("강남역 고기집")

# 검색버튼을 눌러서 결과를 가져옵니다
searchbutton = driver.find_element(By.XPATH, "//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# 검색 결과를 가져올 시간을 기다립니다
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다
html = driver.page_source

# BeautifulSoup을 이용하여 html 정보를 파싱합니다
soup = BeautifulSoup(html, "html.parser")
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []
for moreview in moreviews:
    page_url = moreview.get("href")
    print(page_url)
    page_urls.append(page_url)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://place.map.kakao.com/95713992
https://place.map.kakao.com/2011092566
https://place.map.kakao.com/1503746075
https://place.map.kakao.com/741391811
https://place.map.kakao.com/1648266796
https://place.map.kakao.com/27238067
https://place.map.kakao.com/1052874675
https://place.map.kakao.com/168079537
https://place.map.kakao.com/1111006760
https://place.map.kakao.com/1511737742
https://place.map.kakao.com/263830255
https://place.map.kakao.com/1780387311
https://place.map.kakao.com/56929632
https://place.map.kakao.com/1907052666
https://place.map.kakao.com/12997393


In [83]:
columns = ['score', 'review']
df_shop = pd.DataFrame(columns=columns)
dic_shop = {}

# driver = webdriver.Chrome(path)  # for Mac
driver = webdriver.Chrome(executable_path=path)  # for Windows
for page_url in page_urls:
    
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    time.sleep(2)
    
    # 첫 페이지 리뷰를 크롤링합니다
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
#     contents_div = soup.find(name="div", attrs={"class":"ahead_info"})
    mailshop = soup.find(name="div", attrs={"class":"inner_place"}).find_all('h2')[0].get_text()
    dic_shop[mailshop] = [page_url]
    
    try:
        another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[2]/a/span[1]")
    except:
        try:
            another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[3]/a/span[1]")
        except:
            try:
                another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[8]/div[3]/a/span[1]")
            except:
                continue
    another_reviews.click()
    time.sleep(1)
    # another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[3]/a/span[1]")
    try:
        another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[2]/a/span[1]")
    except:
        try:
            another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[3]/a/span[1]")
        except:
            try:
                another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[8]/div[3]/a/span[1]")
            except:
                continue
    another_reviews.click()
    time.sleep(1)
    try:
        another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[2]/a/span[1]")
    except:
        try:
            another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[7]/div[3]/a/span[1]")
        except:
            try:
                another_reviews = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[2]/div[8]/div[3]/a/span[1]")
            except:
                continue
    another_reviews.click()
    time.sleep(1)    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        contents_list = soup.find(name="div", attrs={"class":"evaluation_review"}).find_all('li')
    except:
        continue
    
    for i, k in enumerate(contents_list):
        # print(i)
        try:
            score = k.find('div', attrs={'class':'unit_info'}).find_all('span', attrs={'class':'txt_desc'})[1].get_text()
        except:
            score = ''
        # print(score)
        try:
            comment = k.find('div', attrs={'class':'comment_info'}).find_all(name="p", attrs={"class":"txt_comment"})[0].get_text() # url
        except:
            comment = ''
        print(score, comment)
        dic_shop[mailshop].append([score, comment])
            
        

4.5 더보기
 
2.7 일단 닭에서 잡내났음물통 입구엔 물때껴있음맛도 양도 무엇하나 특출난건 없었음게다가 휴지까지 셀프인 인건비절감의 교과서근데 사람 많음 ㅋㅋㅋㅋ더보기
 
3.5 더보기
 
4.3 닭갈비는 어디가든 맛이 비슷한거같습니다 양은 푸짐했어요 ~더보기
 
2.3 매장에 날파리가 왤캐 많은지 위생 수준은 훤히 보이고 싸가지없는 직원들 인상 팍팍쓰며 응대하는 꼬락서니보니 밥상 엎어버리고 싶은 맘이 굴뚝. 알바들 교육이 이모냥이니 제대로 된 위생, 식자재 관리가 될 지는 안봐도 유투브. 사흘 굶은 꽃제비들도 공짜로 줘도 안 먹을 음식 내놓곤 3만원 뜯어가는 거 보니 칼든 흉노족. 먹고 나서 배아픈 건 따블. 별 하나 준 것만으로도 성은이 망극하옵니다 외치면서 카카오 본사 방향으로 큰절 세 번 올리고 달궈진 닭갈비 무쇠철판에 이마 세 번 찧어라.더보기
 
3.1 직원 친절이 문제가 아니고그냥 간이 덜배고 맛이 없음 강남역에 위치해서 사람이 많이 가는듯하다맛집은 절대 아님더보기
 
3.6 치즈닭갈비 맛있어요 자주감더보기
 
1.0 장사를 하려는건지 안하려는건지... 공손까지는 바라지도 않고, 손님한테 짜증만 안내도 좋을 것 같습니다.더보기
 
2.3 남자 앞머리 있는 모자쓴 알바생 매우 불친절함 표정도 팔기 싫은데 억지로 일하는 느낌 양도 적고 맛도 짜고 최악더보기
 
3.7 맛있게 잘 먹었습니다더보기
 
 
 
3.9 Good/decent but not amazing. A bit too salty and greasy. Too much cheese.더보기
 
 
 
 
3.4 밑에 사진들 보면 아시겠지만 다른 곳 1인분이 여기 2인분임.. 별점 낮은거 신경쓰였는지 12월 8일자로 다수가 5개를 줬네요?더보기
 
 
3.1 사회적 상황이 어렵게 되면서 다들 힘들게 번 돈으로 방문한 손님들이에요. 저는 오늘 처음 갔고 딱 이거만 같이 먹은 일행 모두 이 새벽에 설사하고 있네요. 우연일수도 있죠.20% 정도는 까맣게 태워 주셔서 탄 맛 때문에 버렸어요. 덕분에 

In [85]:
dic_shop.keys()

dict_keys(['장인닭갈비 강남점', '두껍삼 역삼직영점', '창고43 강남점', '육전식당 4호점', '육품', '두껍삼 강남직영점', '고메램 강남점', '60년전통신촌황소곱창 강남직영점', '원조부안집 강남점', '돝고기506', '강남곱', '국제수입식당 본점', '신도세기 강남역점', '교대이층집 강남역점', '칼맞은삼겹살 강남본점'])